<div class="image-container">
  <img class="image" src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Logo-ITESO-Vertical-SinFondo-png.png" width="300" height="300" style="float: right">
</div>


<p>
    <font style = "font-family:Cambria">          
    <h2><font color="#191970" size = 6 >
        001 Modelo de Puntuación crediticia
    </font></h2>         
    <br>         
    <font color='navy' size = 6>Project 1. </font>
    <br> 
    <h3><font size =4>Collaborators:</font></h3> 
    <ul>
        <li>Diego Emilio Enriquez Nares</li>
        <li>Ana Sofia Brizuela Casarin</li>
        <li>Juan Antonio Mugica Liparoli</li>
    </ul>               
    </font>
</p>